In [1]:

import os

#os.environ['CUDA_VISIBLE_DEVICES'] = '-1' #disble gpu

def get_text_data():
    sentences=[]
    file_name="cleaned_assamese_text.txt"
    file=open(file_name,'r')
    file_sentences=file.read().split(',')
    sentences+=file_sentences
    file.close()
    sentences=list(filter(None,sentences))
    return sentences

sentences=get_text_data()

#maxlen = len(max(sentences))
maxlen=50

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import TextVectorization

def custom_standardization(input_string):
    sentence = tf.strings.lower(input_string)
    #sentence = tf.strings.regex_replace(sentence, "\n", " ")
    return sentence

vectorize_layer = TextVectorization(
    standardize = custom_standardization,
    output_mode="int",
    output_sequence_length=maxlen + 1,
)

vectorize_layer.adapt(sentences)
vocab = vectorize_layer.get_vocabulary()
vocab_size = len(vocab)
index_lookup = dict(zip(range(len(vocab)), vocab))

2023-03-02 12:37:11.424814: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 12:37:12.361581: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.0/lib64:/usr/local/cuda-11.7/lib64::/home/yuvrajtalukdar/miniconda3/envs/miniproject/lib/
2023-03-02 12:37:12.361928: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda-12.0/

In [3]:
import keras_nlp
from tensorflow import keras

embed_dim = 128
num_heads = 4

def create_model2(no_of_decoder=1):
    inputs = keras.layers.Input(shape=(maxlen,), dtype=tf.int32)
    x = keras_nlp.layers.TokenAndPositionEmbedding(vocab_size, maxlen, embed_dim)(inputs)
    for i in range(4):
        x = keras_nlp.layers.TransformerDecoder(intermediate_dim=embed_dim*2, num_heads=num_heads,dropout=0.5)(x)
    do = keras.layers.Dropout(0.4)(x)
    outputs = keras.layers.Dense(vocab_size, activation='softmax')(do)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    model.compile(
        optimizer="adam", 
        loss='sparse_categorical_crossentropy',
        metrics=[keras_nlp.metrics.Perplexity(), 'accuracy']
    )
    return model

model=create_model2(4)

model.load_weights("pd_plaintext_transformer.h5")


2023-03-02 12:37:27.539533: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 155379200 exceeds 10% of free system memory.
2023-03-02 12:37:27.708615: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 155379200 exceeds 10% of free system memory.


In [57]:
def sample_token(logits):
        logits, indices = tf.math.top_k(logits, k=5, sorted=True)
        indices = np.asarray(indices).astype("int32")
        preds = keras.activations.softmax(tf.expand_dims(logits, 0))[0]
        preds = np.asarray(preds).astype("float32")
        return np.random.choice(indices, p=preds)

def generate_text(prompt, response_length=50):
    decoded_sample = prompt
    for i in range(response_length-1):
        tokenized_prompt = vectorize_layer([decoded_sample])[:, :-1]
        predictions = model.predict([tokenized_prompt], verbose=0)
        sample_index = len(decoded_sample.strip().split())-1
        #print(sample_index)
        
        if sample_index>=len(predictions[0]):
            continue

        sampled_token = sample_token(predictions[0][sample_index])
        sampled_token = index_lookup[sampled_token]
        decoded_sample += " " + sampled_token
    return decoded_sample

In [59]:
text='লাচিত বৰফুকনে মোগলৰ বিৰুদ্ধে যুদ্ধ কৰি অসমখন বচাইছিল।'
print(len(text))
generate_text(text,response_length=50)

53


'লাচিত বৰফুকনে মোগলৰ বিৰুদ্ধে যুদ্ধ কৰি অসমখন বচাইছিল। আৰু নিজৰ জীৱন আৰম্ভ কৰিছিল আৰু তেওঁৰ মতে তেওঁ কয়   আৰু আন আন এক গুৰুত্বপূৰ্ণ আৰু তেওঁৰ পত্নী আছিল। কিন্তু তেওঁৰ পত্নী আৰু পুত্ৰ সন্তান আছিল। তেওঁক ভাৰতৰ প্ৰথম অৱস্থাত তেওঁ এজন ব্যক্তিয়ে তেওঁক নিজৰ জীৱন যাপন কৰে বুলি বৰ্ণনা কৰিছিল। সেই সময়ত'